In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings, os, pickle, math, pdb

import matplotlib.pyplot as plt
from vis.utils import utils
from vis.input_modifiers import Jitter

from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks  import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import image
from keras.models import load_model
from keras import backend as K, activations


from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocessor_input
from keras.applications.inception_v3 import decode_predictions as inceptionv3_decode_predictions

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_preprocessor_input
from keras.applications.vgg16 import decode_predictions as vgg16_decode_predictions

from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_preprocessor_input
from keras.applications.vgg19 import decode_predictions as vgg19_decode_predictions

from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_preprocessor_input
from keras.applications.resnet50 import decode_predictions as resnet50_decode_predictions



global model
global modelvgg16
global modelvgg19
global modelinceptionv3
global modelresnet50
global input_width
global input_height
global decode_predictions
global preprocess_input

plt.style.use('ggplot')
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

model = None
modelvgg16 = None
modelvgg19 = None
modelinceptionv3 = None
modelresnet50 = None

Using TensorFlow backend.


In [2]:
def switchvgg16():
    global model
    global modelvgg16
    global input_width
    global input_height
    global decode_predictions
    global preprocess_input
    
    input_width = 224
    input_height = 224
    model = modelvgg16
    preprocess_input = vgg16_preprocessor_input
    decode_predictions = vgg16_decode_predictions
     
def switchvgg19():
    global model
    global modelvgg19
    global input_width
    global input_height
    input_width = 224
    input_height = 224
    model = modelvgg19
    preprocess_input = vgg19_preprocessor_input
    decode_predictions = vgg19_decode_predictions
     
    
def switchinceptionv3():
    global model
    global modelinceptionv3
    global input_width
    global input_height
    global decode_predictions
    global preprocess_input
    
    input_width = 299
    input_height = 299
    model = modelinceptionv3
    preprocess_input = inceptionv3_preprocessor_input
    decode_predictions = inceptionv3_decode_predictions
    
    
def switchresnet50():
    global model
    global modelresnet50
    global input_width
    global input_height
    global decode_predictions
    global preprocess_input
    
    input_width = 224
    input_height = 224
    model = modelresnet50
    preprocess_input = resnet50_preprocessor_input
    decode_predictions = resnet50_decode_predictions
    
    
def loadinceptionv3():
    global modelinceptionv3
    
    print "Loading Inception ... "
    if modelinceptionv3 is None:
        modelinceptionv3 = InceptionV3(weights='imagenet')
    print "Model loaded"
   
    switchinceptionv3()
    
def loadresnet50():
    global modelresnet50
    
    print "Loading Resnet50 ... "
    if modelresnet50 is None:
        modelresnet50 = ResNet50(weights='imagenet')
    print "Model loaded"
    
    switchresnet50()
    
def loadvgg16():
    global modelvgg16

    print "Loading VGG16 ... "
    if modelvgg16 is None:
        modelvgg16 =  VGG16(weights='imagenet')
    print "Model loaded"
    switchvgg16()

def loadvgg19():
    global modelvgg19
    
    print "Loading VGG19 ... "
    if modelvgg19 is None:
        modelvgg19 =  VGG19(weights='imagenet')
    print "Model loaded"
    switchvgg19()
    
    
def doinference(path):
    global model
    img_path = path
    img = image.load_img(img_path, target_size=(input_width, input_height))
    x = image.img_to_array(img)
    x = x.reshape((1, x.shape[0], x.shape[1], x.shape[2]))
    #x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    print('Predicted:',  decode_predictions(preds))
    
    



In [3]:
def vgg():
    vgg = VGG16(weights='imagenet', include_top=False);
    layer_outputs = [layer.output for layer in vgg.layers if 'conv1' in layer.name]
    for layer in vgg.layers:
        print layer.name
    
    activation_model = Model(inputs=vgg.input, outputs=layer_outputs)
    intermediate_activations = activation_model.predict(img_tensor)
    first_layer_activation = intermediate_activations[0]
#plt.imshow(first_layer_activation[0, :, :, 19], cmap='viridis')
#plt.axis('off')
#plt.show()
    for layerll in intermediate_activations:
        showlayer(layerll)

    
def vggpredict():
    model = VGG16(weights='imagenet')
    #model = keras.models.load_model("/home/arshad/.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5")
    img_path = '/home/arshad/projects/Paper/zebra.jpg'
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))

def inception():
    model = InceptionV3(weights='imagenet')
    #model = keras.models.load_model("/home/arshad/.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5")
    img_path = '/home/arshad/projects/Paper/zebra.jpg'
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))
    
def showlayer(layeroutput):
    w=10
    h=10
    
    w = layeroutput.shape[1]
    h = layeroutput.shape[2]
    count =  layeroutput.shape[3]
    print count
    fig=plt.figure(figsize=(w, h))
    columns = int( math.sqrt(count))
    rows = int(math.sqrt(count)) 
    print columns , rows
    for i in range(1, columns*rows +1 ):
        #img = np.random.randint(10, size=(h,w))
        fig.add_subplot(rows, columns, i)
        plt.axis('off')
        plt.imshow(layeroutput[0,:,:, i-1], cmap='gray')
    plt.show(block=False)
    
    
img_path = '/home/arshad/projects/Paper/horse.jpg'


img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)

img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.
print(img_tensor.shape)
#plt.imshow(img_tensor[0])
#plt.axis('off')
#plt.show()

(1, 150, 150, 3)


In [4]:
file='/home/arshad/projects/Paper/zebra.jpg'
loadvgg16()
doinference(file)
loadvgg19()
doinference(file)
loadinceptionv3()
doinference(file)
loadresnet50()
doinference(file)
#pdb.set_trace() #drop to python shell
#plt.style.use('ggplot')
#plt.imshow(img_tensor[0])
#plt.axis('off')
#plt.show()

    
#print "hello"

Loading VGG16 ... 
Instructions for updating:
Colocations handled automatically by placer.
Model loaded
('Predicted:', [[(u'n02391049', u'zebra', 0.99975437), (u'n02423022', u'gazelle', 8.930854e-05), (u'n02422699', u'impala', 7.757987e-05), (u'n02422106', u'hartebeest', 6.339619e-05), (u'n01518878', u'ostrich', 7.049558e-06)]])
Loading VGG19 ... 
Model loaded
('Predicted:', [[(u'n02391049', u'zebra', 0.9999635), (u'n02422106', u'hartebeest', 1.5399242e-05), (u'n02423022', u'gazelle', 1.4861966e-05), (u'n02422699', u'impala', 4.251905e-06), (u'n02397096', u'warthog', 1.2643867e-06)]])
Loading Inception ... 
Model loaded
('Predicted:', [[(u'n02391049', u'zebra', 0.9497816), (u'n02422106', u'hartebeest', 0.001496049), (u'n01518878', u'ostrich', 0.0007340291), (u'n02422699', u'impala', 0.00049439847), (u'n02437312', u'Arabian_camel', 0.00039523802)]])
Loading Resnet50 ... 
Model loaded
('Predicted:', [[(u'n02391049', u'zebra', 0.99970645), (u'n02422106', u'hartebeest', 7.984984e-05), (u'n